# Importing & Installing Libraries

In [1]:
# Mount Project to Googloe Drive
from google.colab import drive
import os, sys
drive.mount('/content/gdrive/')

# Path to Code Folders
data_dir_path = "/content/gdrive/MyDrive/BME_Design/PEAC_Code/WESAD"




Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
# Install Analysis Libraries
!pip install pyhrv
!pip install nolds
!pip install spectrum
!pip install neurokit2
!pip install scipy
!pip install statistics
!pip install tensorflow
!pip install biosppy

In [3]:
# Import libraries
import numpy as np
import pandas as pd  # reading/managing dataset
import matplotlib.pyplot as plt  # plotting
from scipy import signal 
import statistics as stat
import glob
import biosppy
import neurokit2 as nk
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split  


# Data Extraction and Preparation

In [4]:
# --- PATHS TO WESAD DATA FOLDERS --- # 
folder_paths = [
                "/S2/S2",
                "/S3/S3",
                "/S4/S4",
                "/S5/S5",
                "/S6/S6",
                "/S7/S7",
                "/S8/S8",
                "/S9/S9",
                "/S10/S10",
                "/S11/S11",
                "/S13/S13",
                "/S14/S14",
                "/S15/S15",
                "/S16/S16",
                "/S17/S17",
]



In [5]:
# --- VARIABLE DECLARATION FOR DATASET --- # 
sampling_rate = 700

In [6]:
# --- FUNCTION TO EXTRACT EVENT TIMES FOR DATA LABELS --- # 
"""@brief       :   function to extract information from WESAD quest csv
   @params[in]  :   datapath - string path to data folder
   @params[out] :   event_times - tuple list of start and end times for each data label
   @params[out] :   anxiety_times - start and end time tuple for recording of anxiety time
"""
def get_event_times(datapath):
  event_times = []                #< array to hold start and end times of recording events 
  anxiety_times = []              #< array to hold start and end tme stamps fo anxiety events

  # Convert Quest information cav to pandas dataframe
  quest_df = pd.read_csv(datapath + '_quest.csv')

  # Extract Start and end times as strings
  start_times = (((quest_df.iloc[1]).to_numpy()[0]).split(';'))[1:]
  end_times = (((quest_df.iloc[2]).to_numpy()[0]).split(';'))[1:]

  # Extract labels
  state_order = (((quest_df.iloc[0]).to_numpy()[0]).split(';'))[1:]
  order_idx = None

  # Loop through values and pair start & end times
  for val in range(len(start_times)):
    try:
      # Convert Start and end times from string (minutes) to float (sec)
      start = float(start_times[val])*60
      end = float(end_times[val])*60
      event_times.append([start, end])
      # Determine state in which stress readings were recorded
      if str(state_order[val])[0] == str("T"):
        order_idx = val
    except:
      continue

  # Determine start and end times for anxiety events
  if not order_idx == None:
    anxiety_times = event_times[order_idx]

  return event_times, anxiety_times

In [7]:
# --- FUNCTION TO EXTRACT BIOSIGNALS FROM DICTIONARY OF RESPIBAN DATA --- # 
"""@brief       :   function to extract ecg, eda and acc data from pickled respiban dictionary
   @params[in]  :   datapath - string path to data folder
   @params[out] :   ecg_rawdata - array of ecg recording
   @params[out] :   eda_rawdata - array of eda recordings
   @params[out] :   acc_rawdata - array of accelerometer recordings
   @params[out] :   data_len - length of arrays /number of recordings
"""
def extract_respiban_data(datapath):
  # Unpickle respiban data package and convert to dictionary
  data_dic = pd.read_pickle(datapath + '.pkl')
  respiban_dic = data_dic['signal']['chest']

  ecg_rawdata = respiban_dic['ECG']
  eda_rawdata = respiban_dic['EDA']
  acc_rawdata = respiban_dic['ACC']

  data_len = ecg_rawdata.shape[0]

  return ecg_rawdata, eda_rawdata, acc_rawdata, data_len

In [8]:
# --- FUNCTION TO FILTER ECG & EDA DATA --- # 
"""@brief       :   function to filter ECG and EDA data
   @params[in]  :   ecg - raw ecg data
   @params[in]  :   eda - raw eda data 
   @params[in]  :   sampling_rate - sampling rate of the data 
   @params[out] :   ecg_filtered  - filtered ecg data
   @params[out] :   eda_filtered  - filtered eda data
"""
def filter_data(ecg,eda,sampling_rate):
  # ECG Filtering
  ecg_rawdata_features = biosppy.signals.ecg.ecg(signal=ecg, sampling_rate=sampling_rate)
  ecg_filtered = ecg_rawdata_features['filtered']

  # EDA Filtering
  signals ,_ = nk.eda_process(eda,sampling_rate = sampling_rate)
  eda_filtered = signals['EDA_Clean']

  return ecg_filtered, eda_filtered 

In [9]:
# --- FUNCTION TO CONSTRUCT ARRAY OF TIME SERIES DATA --- # 
"""@brief       :   function to create an array of time values for respiban recordings
   @params[in]  :   data_len - number of datapoints in each recording array
   @params[in]  :   sampling_rate
   @params[out] :   end_time  - timestamp of last recording
   @params[out] :   dt  - timestep between recordings
   @params[out] :   t - array of time series data
"""
def get_time_data(data_len, sampling_rate):
  end_time = data_len/sampling_rate

  t = np.linspace(0, end_time, data_len)

  dt = t[0] - t[1]

  return t

In [10]:
# --- FUNCTION TO ADD LABELS TO DATA --- # 
"""@brief       :   function to create an array of time values for respiban recordings
   @params[in]  :   data_len - number of datapoints in each recording array
   @params[in]  :   sampling rate  - data sampling rate
   @params[in]  :   t - array of time series data
   @params[in]  :   anxiety_time - start and end time of anxiety event
   @params[out] :   labels - array of labeled data for specifi time points
                      0 = no anxiety, 1 = anxiety
"""
def label_data(data_len, sampling_rate, t, anxiety_time):
  # create array to store label data
  labels = np.zeros(data_len)

  # determine indexes for start and end of anxiety events
  start = int(anxiety_time[0]*sampling_rate)
  end = int(anxiety_time[1]*sampling_rate)

  # add labels
  labels[start:end] = 1

  return labels



In [11]:
# --- FUNCTION TO CREATE DATA FRAME FOR EACH PARTICIPANT / TIME SERIES --- # 
"""@brief       :   function to create a dataframe to store labelled biosignal data for each participant
   @params[in]  :   ecg_data 
   @params[in]  :   eda_data
   @params[in]  :   acc_data
   @params[in]  :   time_data
   @params[in]  :   labels
   @params[out] :   s_df - structured datatframe consisting of a particpant's recorded data 
"""
def create_subject_df(ecg_data, eda_data, acc_data, time_data, labels):


  s_dic = {
    'TIME'          : time_data,
    'EDA_RESPIBAN'  : eda_data,
    'ECG_RESPIBAN'  : ecg_data,
    'X_RESPIBAN'    : acc_data[0],
    'Y_RESPIBAN'    : acc_data[1],
    'Z_RESPIBAN'    : acc_data[2],
    'LABEL'         : labels
  }

  # Convert dictionary into dataframe
  s_df = pd.DataFrame.from_dict(s_dic)

  return s_df







In [12]:
# --- MAIN  --- # 


# Loop through folders and extrapolate data
for path in folder_paths:
  #Define Datapath to acess subject recordings
  datapath = data_dir_path + path
  print(path)

  # Extract event times from quest csv
  event_times, anxiety_times = get_event_times(datapath)
  print(anxiety_times)

  # Unpack and filter respiban data
  ecg_rawdata, eda_rawdata, acc_rawdata, data_len = extract_respiban_data(datapath)
  #ecg_filtered, eda_filtered = filter_data(ecg_rawdata, eda_rawdata, sampling_rate)
  # Adjust numpy dimensions
  ecg_rawdata = ecg_rawdata.flatten()
  eda_rawdata = eda_rawdata.flatten()
  acc_rawdata = np.transpose(acc_rawdata)

  # Create array of time data
  t = get_time_data(data_len, sampling_rate)

  # Create array to store data labels
  labels = label_data(data_len, sampling_rate, t, anxiety_times)

  # Store Data for each participant in dictionary
  s_df = create_subject_df(ecg_rawdata, eda_rawdata, acc_rawdata, t, labels)




/S2/S2
[2373.0, 3018.0]
/S3/S3
[2289.0, 2949.0]
/S4/S4
[3672.0, 4329.0]
/S5/S5
[3660.0, 4323.0]
/S6/S6
[2469.0, 3135.0]
/S7/S7
[3126.0, 3786.0]
/S8/S8
[3366.0, 4044.0000000000005]
/S9/S9
[1887.0, 2550.0]
/S10/S10
[3258.0, 3993.0]
/S11/S11
[1947.0000000000002, 2655.0]
/S13/S13
[3312.6, 3987.0]
/S14/S14
[1990.8, 2671.8]
/S15/S15
[3248.4, 3960.0]
/S16/S16
[2058.0, 2761.8]
/S17/S17
[3601.2000000000003, 4335.0]


In [13]:
x = s_df.iloc[:,1:6].values
y = s_df.iloc[:,-1].values

# Signal Processing, Pre-processing and Feature Scaling

In [14]:
# --- FUNCTION TO SEGMENT THE DATA INTO TIME STEPS --- # 
"""@brief       :   function to break the data into segments based on the timestep input
   @params[in]  :   x_traning - filtered training data 
   @params[in]  :   y_traning - filtered training data 
   @params[in]  :   timestep - number of previous obesevations to be considered when the NN makes an observation 
   @params[out] :   x_train  - training data in a 2D numpy array
   @params[out] :   y_train  - training data labels 
"""
def segment_data(x_training,y_training,timestep):
  x_train = []
  y_train = []
  for i in range(timestep, len(x_training)):

    x_train.append(x_training[i-timestep:i, :])

    y_train.append(y_training[i])
  
  x_train = np.array(x_train)
  y_train = np.array(y_train)

  return x_train, y_train

In [15]:
# --- FUNCTION TO CONVERT ALL OF THE DATA --- # 
"""@brief       :   function to do all of the data conversions and filtering 
   @params[in]  :   ecg_signal - raw ecg signal 
   @params[in]  :   eda_signal - raw eda signal
   @params[in]  :   acc_signal - raw accelertaion signal
   @params[in]  :   timestep - number of previous obesevations to be considered when the NN makes an observation 
   @params[out] :   x_training  - training data in a 2D numpy array
   @params[out] :   y_training  - training data labels 
"""

'@brief       :   function to do all of the data conversions and filtering \n   @params[in]  :   ecg_signal - raw ecg signal \n   @params[in]  :   eda_signal - raw eda signal\n   @params[in]  :   acc_signal - raw accelertaion signal\n   @params[in]  :   timestep - number of previous obesevations to be considered when the NN makes an observation \n   @params[out] :   x_training  - training data in a 2D numpy array\n   @params[out] :   y_training  - training data labels \n'

In [16]:
# --- FUNCTION TO SPLIT THE DATAFRAME INTO TRAINING, TEST, & VALIDATION DATA --- #
"""@brief       :   function to split the data into 
   @params[in]  :   x - x section of the dataframe
   @params[in]  :   y - y section of the dataframe
   @params[in]  :   frac1 - fraction of the data to store as test/validation data
   @params[in]  :   frac2 - fraction of the data to store from test/validation data
   @params[out] :   x_train  - x training data 
   @params[out] :   y_train - y training data
   @params[out] :   x_test  - x test data 
   @params[out] :   y_test - y test data 
   @params[out] :   x_val  - x validation data 
   @params[out] :   y_test - y validation data 
"""
def split_data(x,y,frac1,frac2):

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=frac1)
  x_test, x_val, y_test, y_val = train_test_split(x_test,y_test, test_size =frac2)

  return x_train,y_train,x_test,y_test, x_val, y_val

# Model Construction

In [ ]:
scaler = MinMaxScaler()


# Hyperparameter Tuning

# Model Evaluation

#Main

In [ ]:
# Seperate the data into training, test, and validation
frac1 = .4 # 40% of data store for test/validation
frac2 = .5 # 50% of that 40% (so 20% of origional dataset) stored as validation data 
[x_training,y_training,x_test,y_test,x_val,y_val] = split_data(x,y,frac1,frac2)

timestep = 700
# Segment the data into timestep
[x_train,y_train] = segment_data(x_training,y_training,timestep)
# may have to reshape the data 

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],  x_train.shape[1], 1))

In [ ]:
# RNN
rnn = Sequential()
# specify number of neurons desired in LSTM layer 
units = 45 # given from an example online (will likely change)
return_sequence = True #unless it is the last LSTM Layer in the model then it is False
# specify the shape of the training data 
input_shape = x_train.shape[1]

for i in [True, True, False]:

    rnn.add(LSTM(units, return_sequences = i))

    rnn.add(Dropout(0.2))
# Output layer    
rnn.add(Dense(units = 1))

# Compile the RNN
# May want to change the optimizer or loss here 
rnn.compile(optimizer = 'adam', loss = 'mean_squared_error')


In [ ]:
# Fit the RNN to the training data 
# specify epochs 
epochs = 10 # will change 
# specify batch_size
batch_size = 2 # will change

rnn.fit(x_train,y_train,epochs,batch_size)

Epoch 1/2
  1796/248620 [..............................] - ETA: 12:36:05 - loss: 0.1065

KeyboardInterrupt: ignored

In [ ]:
# reshape test data 
x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], 

                              x_test.shape[1], 

                                          1))